In [23]:
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import RMSprop
from keras import Model

In [24]:
BATCH_SIZE = 16

weights_path = 'vgg16_weights.h5'

WIDTH, HEIGHT = 224, 224

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'flower_photos/train',  # this is the target directory
        target_size=(WIDTH, HEIGHT),  # all images will be resized to 150x150
        batch_size=BATCH_SIZE,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'flower_photos/test',
        target_size=(WIDTH, HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='categorical')



Found 2982 images belonging to 5 classes.
Found 687 images belonging to 5 classes.


In [25]:
TRAIN_SIZE = 2982
VAL_SIZE = 687
EPOCHS = 50
CLASSES = 5

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(WIDTH, HEIGHT, 3))

top_model = Sequential()
tf_shape = base_model.output_shape[1:]
tf_shape = tf_shape[::-1]
top_model.add(Flatten(input_shape=tf_shape))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.25))
top_model.add(Dense(CLASSES, activation='sigmoid'))


# top_model.load_weights(top_model_weights_path)

model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

for layer in base_model.layers[:25]:
    layer.trainable = False

opt = RMSprop(lr=0.0003)
model.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])

In [26]:
checkpoint = ModelCheckpoint('trained_flowers.h5', monitor='val_loss', verbose=0, save_best_only=True)
model.fit_generator(train_generator,
                    steps_per_epoch=TRAIN_SIZE//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=validation_generator,
                    validation_steps=VAL_SIZE//BATCH_SIZE,
                    callbacks=[checkpoint])

Epoch 1/50


  1/186 [..............................] - ETA: 2:21 - loss: 1.9086 - acc: 0.1250



  2/186 [..............................] - ETA: 1:27 - loss: 2.1535 - acc: 0.1875



...



183/186 [============================>.] - ETA: 0s - loss: 0.0570 - acc: 0.9802

184/186 [============================>.] - ETA: 0s - loss: 0.0567 - acc: 0.9803

185/186 [============================>.] - ETA: 0s - loss: 0.0564 - acc: 0.9804

186/186 [==============================] - 43s 232ms/step - loss: 0.0562 - acc: 0.9805 - val_loss: 0.7889 - val_acc: 0.8557


In [46]:
img = image.load_img('rose.jpg', target_size=(WIDTH, HEIGHT))
img = image.img_to_array(img)/255
img = np.expand_dims(img, axis=0)
print(train_generator.class_indices) 

np.argmax(model.predict(img))


{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


2